## CS 6120: Predicting Retraction Paper

## Importing the Libraries

This is the batch version of Jupyter notebook in the Github repository. Refer to that if you have questions on the code functionality.

In [13]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import warnings
import seaborn as sns
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")
from sklearn.utils import resample
import string
import math
import os
import glob
import statistics
import csv

warnings.filterwarnings("ignore")

In [14]:
path = 'C:\\Users\\bspia\\Documents\\Jupyter\\NLP_Project_Jupyter_Retraction_Reason'
extension = 'csv'
os.chdir(path)
filelist = glob.glob('*.{}'.format(extension))
strings = [string[12:-4]for string in filelist]

## Reading the data

 

In [15]:
iterations = 5;

max_feature_count = 500;

stat_array = []

fields = ['RR', 'RF ACC', 'NB ACC', 'LR ACC']

for k in range(len(filelist)):
    
    print(k)
    
    rf_acc_array = [];
    nb_acc_array = [];
    lr_acc_array = [];

    for i in range(iterations):
        ## Reading the data and removing columns that are not important. 
        df = pd.read_csv(filelist[k], sep = ',', encoding = 'latin-1', usecols = lambda col: col not in ["Unnamed: 0"])

        from sklearn.utils import resample

        df_majority = df[df.Target==0]
        df_minority = df[df.Target==1]

        spam_upsample = resample(df_minority, replace = True, 
                                n_samples = df_majority.shape[0],
                                random_state = 101)

        df_upsampled = pd.concat([df_majority, spam_upsample])  
        df_upsampled = df_upsampled.sample(frac = 1)

        from sklearn.feature_extraction.text import CountVectorizer
        from sklearn.model_selection import train_test_split,StratifiedKFold
        sentences = df_upsampled['Text'].tolist()
        vectorizer = CountVectorizer(max_features = max_feature_count)
        X = vectorizer.fit_transform(sentences)        
        y = df_upsampled['Target'].tolist()
        X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.8, stratify = y, random_state = 123)

        ## Random Forest

        from sklearn.ensemble import RandomForestClassifier
        from sklearn.metrics import confusion_matrix
        from sklearn.metrics import accuracy_score
        from sklearn.metrics import classification_report

        m1 = 'Random Forest Classfier'
        rf = RandomForestClassifier(n_estimators=50, random_state=12,max_depth=3)
        rf.fit(X_train,y_train)
        rf_predicted = rf.predict(X_test)
        rf_conf_matrix = confusion_matrix(y_test, rf_predicted)
        rf_acc_score = accuracy_score(y_test, rf_predicted)

        from sklearn.naive_bayes     import MultinomialNB
        # initialize a Naive bayes model
        m2 = 'Naive Bayes Classfier'
        naive      = MultinomialNB()

        # Fit the model using the training data
        nb = naive.fit(X_train,y_train)
        # predict the author of the held-out test sentences
        nb_predicted = nb.predict(X_test)
        nb_conf_matrix = confusion_matrix(y_test, nb_predicted)
        nb_acc_score = accuracy_score(y_test, nb_predicted)

        from sklearn.feature_extraction.text import CountVectorizer
        from sklearn.linear_model    import LogisticRegression
        from sklearn.model_selection import train_test_split,StratifiedKFold
        from sklearn.metrics         import confusion_matrix

        #-------------------------------------------------------
        # Initialize four Logistic Regression models, 
        # each with a different penalty
        #-------------------------------------------------------
        lr        = LogisticRegression(penalty  = 'none')                                                # No penalty
        lr_l2     = LogisticRegression(penalty  = 'l2'        , solver = 'newton-cg' )                   # L2 penalty
        lr_l1     = LogisticRegression(penalty  = 'l1'        , solver = 'saga'      )                   # L1 penalty
        lr_l1l2   = LogisticRegression(penalty  = 'elasticnet', solver = 'saga' ,l1_ratio = 0.5)         # L1-L2 penalty

        #-------------------------------------------------------
        # Fit all four models using the training data
        # Store a list of models for ease of evaluation later...
        #-------------------------------------------------------
        models = {}
        models['Logistic_Regression']              = lr.fit(X_train,y_train)
        #models['Logistic_Regression_(L2)']         = lr_l2.fit(X_train,y_train)
        #models['Logistic_Regression_(L1)']         = lr_l1.fit(X_train,y_train)
        #models['Logistic_Regression_(Elasticnet)'] = lr_l1l2.fit(X_train,y_train)

        lr_predict = lr.predict(X_test)
        lr_conf_matrix = confusion_matrix(y_test, lr_predict)
        lr_acc_score = accuracy_score(y_test, lr_predict)

        rf_acc_array.append(rf_acc_score*100)
        nb_acc_array.append(nb_acc_score*100)
        lr_acc_array.append(lr_acc_score*100)

    rf_mean = statistics.mean(rf_acc_array)
    rf_std = statistics.stdev(rf_acc_array)
    nb_mean = statistics.mean(nb_acc_array)
    nb_std = statistics.stdev(nb_acc_array)
    lr_mean = statistics.mean(lr_acc_array)
    lr_std = statistics.stdev(lr_acc_array)
    stat_array.append([strings[k],str(rf_mean)+'+-'+str(rf_std),str(nb_mean)+'+-'+str(nb_std),str(lr_mean)+'+-'+str(lr_std)])

rows = stat_array

with open('Results_Reasons.csv', 'w') as f:
      
    # using csv.writer method from CSV package
    write = csv.writer(f)
      
    write.writerow(fields)
    write.writerows(rows)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
